In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [ ]:
from datasets import load_dataset
import json

def load_and_clean_dataset():
    dataset = load_dataset("ivelin/ui_refexp_saved", split="train[:336]")
    # df = dataset.to_pandas()
    # unique_df = df.drop_duplicates(subset="image_id")
    # cleaned_data = unique_df["image"]
    cleaned_data = []

    for entry in dataset:
        # Check if entry is a string and attempt to convert to dict
        if isinstance(entry, str):
            try:
                entry = json.loads(entry)
            except json.JSONDecodeError:
                print(f"Warning: Failed to decode JSON string: {entry}")
                continue  # Skip this entry if decoding fails

        cleaned_entry = {
            "image": entry.get("image"),  # Use get() to avoid KeyError if key is missing
            "image_id": entry.get("image_id")
        }
        cleaned_data.append(cleaned_entry)
        

    return cleaned_data

In [ ]:
import pandas as pd
prompts1 = pd.read_csv("/kaggle/input/prompts2/prompts - Sheet1.csv")
cleaned = load_and_clean_dataset()
cleaned_df = pd.DataFrame(cleaned)
cleaned_df = cleaned_df.drop_duplicates(subset="image_id", keep="first")
cleaned_df= cleaned_df.drop(columns=['image_id'])
assert len(cleaned_df) == len(prompts1), "Lengths do not match!"
cleaned_df = cleaned_df.reset_index(drop=True)
prompts1 = prompts1.reset_index(drop=True)
cleaned_df["prompts"] = prompts1
cleaned_df.head()

In [ ]:
print(len(cleaned_df))

In [ ]:
import matplotlib.pyplot as plt

# Display the first image
plt.imshow(cleaned_df.iloc[99]["image"])
plt.axis("off")  # Hide axes
plt.show()
print(cleaned_df.iloc[99]["prompts"])

In [ ]:
import importlib.metadata
print(importlib.metadata.version("bitsandbytes"))

In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration, BitsAndBytesConfig

model_id = "llava-hf/llava-v1.6-mistral-7b-hf"

# Set up 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

processor = LlavaNextProcessor.from_pretrained(model_id)

# Load model with quantization and device mapping
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

In [ ]:
# Prepare image (ensure correct mode and size)
# image = cleaned_df.iloc[8]['image']
import pandas as pd

VLMresponses = []

for i in range(len(cleaned_df.index)):

    image = cleaned_df.iloc[i]['image']
    
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    datasetPrompt = cleaned_df.iloc[i]['prompts']
   
    prompt = (
        f"USER: You are a virtual assistant helping a user interact with a mobile app. You are given only this screenshot of a mobile application.\n"
        f"<image>\n"
        f"Do not make assumptions beyond what is clearly visible in the image.\n"
        f"Identify all interactable UI elements shown, such as buttons,icons (share, close, menu, heart, search, etc.), text fields, checkboxes, sliders, and other actionable components.\n"
        f"The user wants to: '{datasetPrompt}'\n"
        f"Using only the current screenshot, return the shortest sequence of taps required to complete this task.\n"
        f"Only include necessary steps visible in the image. Limit it to 1-5 steps.\n"
        f"ASSISTANT:"
    )
    
    #f"The task may require additional interactions with elements not shown on the current screen.\n"
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200, do_sample=True, top_p=0.9, temperature=0.7)
    response = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    assistant_response = response.split("ASSISTANT:")[-1].strip()
    VLMresponses.append({
        "Index": i,
        "UserPrompt": datasetPrompt,
        "AssistantResponse": assistant_response
    })
    print(assistant_response)

In [ ]:
print(VLMresponses[0])

In [ ]:
# Create DataFrame and save to CSV
VLMres_df = pd.DataFrame(VLMresponses)
VLMres_df.to_csv("/kaggle/working/VLM_responses.csv", index=False)